In [1]:
import asyncio
import json
import art
from art.local import LocalBackend
from dotenv import load_dotenv
import openai

load_dotenv()

/workspace/codenames-rl/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-13 07:53:32 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 07:53:32 [__init__.py:239] Automatically detected platform cuda.


2025-05-13 07:53:36,221	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


True

In [18]:
async def rollout(model: art.TrainableModel, prompt: str) -> art.Trajectory:
    messages: art.Messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    client = model.openai_client()
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name, max_tokens=100, timeout=100, extra_body={"chat_template_kwargs": {"enable_thinking": False}},
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    print(content)
    assert isinstance(content, str)
    if content == "yes":
        reward = 0.5
    elif content == "no":
        reward = 0.75
    elif content == "maybe":
        reward = 1.0
    else:
        reward = 0.0
    return art.Trajectory(messages_and_choices=[*messages, choice], reward=reward)

In [6]:
with open("prompts.json", "r") as f:
    prompts = json.load(f)
print(prompts)

["respond with 'yes', 'no', 'maybe'", "respond with 'maybe', 'yes', 'no'", "respond with 'no', 'yes', 'maybe'", "respond with 'yes', 'maybe', 'no'", 'respond with yes or no', 'respond with maybe or no', 'respond with no or maybe', 'respond with no or yes', 'respond with yes or no', 'respond with yes, no, maybe', 'respond with maybe, yes, no', 'respond with no, yes, maybe', 'respond with yes, maybe, no', 'respond with yes or no', 'respond with maybe or no', 'respond with no or maybe', 'respond with no or yes', 'respond with yes or no', "just respond with 'yes', 'no', 'maybe'", "just respond with 'maybe', 'yes', 'no'", "just respond with 'no', 'yes', 'maybe'", "just respond with 'yes', 'maybe', 'no'", 'just respond with yes or no', 'just respond with maybe or no', 'just respond with no or maybe', 'just respond with no or yes', 'just respond with yes or no', 'just respond with yes, no, maybe', 'just respond with maybe, yes, no', 'just respond with no, yes, maybe', 'just respond with yes, 

In [7]:
backend = LocalBackend()

In [8]:
qwen2 = art.TrainableModel(
    name="004",
    project="yes-no-maybe-s",
    base_model="Qwen/Qwen2.5-0.5B-Instruct",
    # base_model="Qwen/Qwen2.5-0.5B-Instruct",
)
await qwen2.register(backend)

/workspace/ART/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 07:55:12 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 07:55:12 [__init__.py:239] Automatically detected platform cuda.
Standard import failed for UnslothBCOTrainer: No module named 'UnslothBCOTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.5.1: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-0.5b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.33it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.21it/s]



INFO 05-13 07:55:49 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-13 07:55:50 [model_runner.py:1140] Model loading took 0.5153 GiB and 1.320794 seconds
INFO 05-13 07:55:52 [worker.py:287] Memory profiling takes 2.51 seconds
INFO 05-13 07:55:52 [worker.py:287] the current vLLM instance can use total_gpu_memory (79.20GiB) x gpu_memory_utilization (0.78) = 62.10GiB
INFO 05-13 07:55:52 [worker.py:287] model weights take 0.52GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 2.07GiB; the rest of the memory reserved for KV Cache is 59.36GiB.
INFO 05-13 07:55:53 [executor_base.py:112] # cuda blocks: 324184, # CPU blocks: 32768
INFO 05-13 07:55:53 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 158.29x
INFO 05-13 07:55:56 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CL

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:30<00:00,  1.63it/s]


INFO 05-13 07:56:26 [model_runner.py:1592] Graph capturing finished in 30 secs, took 0.84 GiB
INFO 05-13 07:56:26 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 36.86 seconds
Unsloth: Just some info: will skip parsing ['q_norm', 'post_feedforward_layernorm', 'pre_feedforward_layernorm', 'k_norm']
Unsloth: Just some info: will skip parsing ['q_norm', 'post_feedforward_layernorm', 'pre_feedforward_layernorm', 'k_norm']


Unsloth 2025.5.1 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [12]:
await rollout(qwen2, prompts[4])

Yes


Trajectory(messages_and_choices=[{'content': 'respond with yes or no', 'role': 'user'}, Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='token_id:9454', bytes=[89, 101, 115], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:151645', bytes=[], logprob=-0.6409199237823486, top_logprobs=[])], refusal=None), message=ChatCompletionMessage(content='Yes', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], tools=None, reward=0.0, metrics={}, metadata={}, logs=[], start_time=datetime.datetime(2025, 5, 13, 7, 57, 45, 484026))

In [14]:
qwen3 = art.TrainableModel(
    name="005",
    project="yes-no-maybe-s",
    base_model="Qwen/Qwen3-0.6B",
    # base_model="Qwen/Qwen2.5-0.5B-Instruct",
)
await qwen3.register(backend)

/workspace/ART/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 08:00:16 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 08:00:17 [__init__.py:239] Automatically detected platform cuda.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!
Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/qwen3-0.6b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - igno

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.84it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.83it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.31it/s]



INFO 05-13 08:00:53 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-13 08:00:53 [model_runner.py:1140] Model loading took 0.5271 GiB and 1.011840 seconds
INFO 05-13 08:00:55 [worker.py:287] Memory profiling takes 1.93 seconds
INFO 05-13 08:00:55 [worker.py:287] the current vLLM instance can use total_gpu_memory (79.20GiB) x gpu_memory_utilization (0.78) = 62.10GiB
INFO 05-13 08:00:55 [worker.py:287] model weights take 0.53GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 2.07GiB; the rest of the memory reserved for KV Cache is 59.34GiB.
INFO 05-13 08:00:56 [executor_base.py:112] # cuda blocks: 34722, # CPU blocks: 3510
INFO 05-13 08:00:56 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 16.95x
INFO 05-13 08:00:59 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:32<00:00,  1.51it/s]


INFO 05-13 08:01:31 [model_runner.py:1592] Graph capturing finished in 32 secs, took 1.16 GiB
INFO 05-13 08:01:31 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 37.90 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


Unsloth 2025.5.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
await rollout(qwen3, prompts[4])

Yes.


Trajectory(messages_and_choices=[{'content': 'respond with yes or no', 'role': 'user'}, Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='token_id:9454', bytes=[89, 101, 115], logprob=-0.0793837234377861, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:13', bytes=[46], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='token_id:151645', bytes=[], logprob=0.0, top_logprobs=[])], refusal=None), message=ChatCompletionMessage(content='Yes.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], tools=None, reward=0.0, metrics={}, metadata={}, logs=[], start_time=datetime.datetime(2025, 5, 13, 8, 3, 38, 210520))

: 

In [ ]:
for _ in range(await model.get_step(), 1_000):
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(rollout(model, prompt) for _ in range(32))
            for prompt in prompts
        ),
        pbar_desc="gather",
    )
    await model.train(
        train_groups,
        config=art.TrainConfig(learning_rate=1e-4),
    )